In [ ]:
import datasets
import torch
import transformers
from transformers import set_seed

from alignment import (
    DataArguments,
    H4ArgumentParser,
    ModelArguments,
    SFTConfig,
    apply_chat_template,
    get_checkpoint,
    get_datasets,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    get_tokenizer,
)
from trl import SFTTrainer

In [2]:
from datasets import load_dataset

dataset = load_dataset("imdb", split="train")


Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 185916.41 examples/s]


In [4]:
print(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [7]:
type(dataset)

datasets.arrow_dataset.Dataset

In [6]:
dataset['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [28]:
import numpy as np
img_tensor = np.zeros((100, 512, 512))
captions = []
for i in range(100):
  captions.append('hello')

captioned_imgs = {
    'images': img_tensor,
    'text': captions
}
from datasets import Dataset
out = Dataset.from_dict(captioned_imgs)
print(out)

Dataset({
    features: ['images', 'text'],
    num_rows: 100
})


In [15]:
import pandas as pd
train_df = pd.read_csv('/home/seungbinyang/LLM-train/alignment-handbook/data/train.csv')
test_df = pd.read_csv('/home/seungbinyang/LLM-train/alignment-handbook/data/test.csv')
train_df.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


In [17]:
test_df.head()

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?


In [16]:
len(train_df)

644

In [49]:
import random
random.seed(42)
q_a = []
category_list = []

for i in range(len(train_df)):
    row = train_df.iloc[i]
    category_list.append(row['category'])

In [50]:
category_cnt = {}
for c in category_list:
    if c in category_cnt:
        category_cnt[c] += 1
    else:
        category_cnt[c] = 1
category_cnt

{'건축구조': 31,
 '기타': 27,
 '마감재': 272,
 '마감하자': 60,
 '시공': 111,
 '인테리어': 123,
 '타 마감하자': 20}

In [51]:
cnt = {k:0 for k in category_cnt.keys()}

train_questions = []
train_answers = []
test_questions = []
test_answers = []

for i in range(len(train_df)):
    row = train_df.iloc[i]
    q1 = row['질문_1']
    q2 = row['질문_2']
    a1 = row['답변_1']
    a2 = row['답변_2']
    a3 = row['답변_3']
    a4 = row['답변_4']
    a5 = row['답변_5']

    q_l = [q1, q2]
    a_l = [a1, a2, a3, a4, a5]

    category = row['category']

    if cnt[category] < category_cnt[category] * 0.8:
        cnt[category] += 1
        for k in range(2):
            for t in range(5):
                train_questions.append(q_l[k])
                train_answers.append(a_l[t])
    else:
        for k in range(2):
            for t in range(5):
                test_questions.append(q_l[k])
                test_answers.append(a_l[t])


In [53]:
len(train_df) * 10

6440

In [52]:
len(train_questions), len(train_answers),len(test_questions), len(test_answers)

(5170, 5170, 1270, 1270)

In [54]:
import json

train_dataset = {'data':[]}
for q, a in zip(train_questions, train_answers):
    train_dataset['data'].append({'question':q,'answer':a})

with open("../data/train.json", "w") as json_file:
    json.dump(train_dataset, json_file, ensure_ascii=False, indent=4)


test_dataset = {'data':[]}
for q, a in zip(test_questions, test_answers):
    test_dataset['data'].append({'question':q,'answer':a})

with open("../data/test.json", "w") as json_file:
    json.dump(test_dataset, json_file, ensure_ascii=False, indent=4)

In [61]:
with open("../data/test.json", "r") as d:
    test_data = json.load(d)

test_texts = []

for data in test_data['data']:
    test_texts.append(f"{data['question']}</s>{data['answer']}")

out = Dataset.from_dict({'texts':test_texts})
print(out['texts'][0])

통나무구조 방식이 뭐야?</s>통나무구조 방식은 원형 또는 각형의 내력벽으로 하고 나머지 바닥이나 지붕구조는 2x4경량 목구조와 동일한 구조로 구성된다. 벽체가 통나무를 쌓는 구조이기 때문에 1개층 높이에서 7~12cm의 침하가 장기적으로 발생하게 되므로 창문틀 등의 개구부에는 침하를 고려한 디테일을 만들어야 합니다.
